### Import the library

In [1]:
import torch
import pickle

from torch.utils.data import Dataset, DataLoader, random_split
from sklearn.model_selection import train_test_split
from torch_geometric.data import Data

In [2]:

from base_function import *

### Data Preparation

In [4]:
pickle_path = r'A:\Code\deepso\parameter_dataset.pkl'
with open(pickle_path, 'rb') as f:
    
    all_functions_by_dim = pickle.load(f)

In [5]:
train_data_combined = []
test_data_combined = []

for d, func_dict in all_functions_by_dim.items():
    for func_type, func_list in func_dict.items():
        if func_type == "largeman" or func_type == "crossintray" or func_type == "dropwave" :
            continue
        
        train_funcs, test_funcs = train_test_split(func_list, test_size=0.3, random_state=42)

        train_data_combined.extend([{'dim': d, 'func_type': func_type, 'params': params} for params in train_funcs])
        test_data_combined.extend([{'dim': d, 'func_type': func_type, 'params': params} for params in test_funcs])

print('split done')


split done


In [6]:
from collections import Counter

def count_functions_by_dim(data_combined, dim_value):
    # Tạo một Counter để đếm số lượng hàm của từng func_type với dim = dim_value
    func_count_by_type = Counter()
    
    # Duyệt qua từng phần tử trong list
    for item in data_combined:
        # Kiểm tra nếu dim bằng giá trị mong muốn
        if item['dim'] == dim_value:
            func_count_by_type[item['func_type']] += 1
    
    return func_count_by_type

# Đếm số hàm có dim = 1 trong train_data_combined
train_func_count = count_functions_by_dim(train_data_combined, 1)
print("Số lượng hàm có dim = 1 trong train_data_combined:", train_func_count)

# Đếm số hàm có dim = 1 trong test_data_combined
test_func_count = count_functions_by_dim(test_data_combined, 1)
print("Số lượng hàm có dim = 1 trong test_data_combined:", test_func_count)


Số lượng hàm có dim = 1 trong train_data_combined: Counter({'ackley': 700, 'griewank': 700, 'levy': 700, 'rastrigin': 700})
Số lượng hàm có dim = 1 trong test_data_combined: Counter({'ackley': 300, 'griewank': 300, 'levy': 300, 'rastrigin': 300})


In [7]:
splited_data = {'train': train_data_combined, 'test': test_data_combined}
splited_path = r'A:\Code\deepso\splited_data_no.pkl'
with open(pickle_path, 'wb') as f:
    pass
with open(splited_path, 'wb') as f:
    pickle.dump(splited_data, f)

print(f"All functions split into combined train and test sets and saved to '{splited_path}'")

All functions split into combined train and test sets and saved to 'A:\Code\deepso\splited_data_no.pkl'


In [8]:
def create_batches(dataset, batch_size): 
    random.shuffle(dataset)
    for i in range(0, len(dataset), batch_size): 
        batch = dataset[i:i + batch_size]
        yield batch